This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'Ill never tell'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
specify = {'start_date': '2017-04-28', 'end_date': '2017-04-28', 'api_key': API_KEY}
rq = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X', params=specify)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(rq.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2019-05-02T22:32:14.090Z', 'newest_available_date': '2019-05-02', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-04-28', 'end_date': '2017-04-28', 'data': [['2017-04-28', 42.17, 42.25, 41.66, 41.72, None, 86348.0, 3606589.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1 - collect data for all of 2017
specify_2017 = {'start_date': '2017-01-01', 'end_date': '2017-12-31', 'api_key': API_KEY}
rq_2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X', params=specify_2017)

In [6]:
#2 - covert object to python dictionary
data_2017 = rq_2017.json()

type(data_2017)

dict

The dictionary has been set up. 

##### Task \#3: highest and lowest opening prices

In [7]:
#find the index for the 'Open' column
data_2017['dataset']['column_names'].index('Open')

1

In [8]:
#extract data for each day
daily = data_2017['dataset']['data']

#now there is a list of lists for each active day stored in the daily variable
#and the Open price for each list is stored at index 1. Now we can create a list of all Open prices
opens = [day[1] for day in daily if day[1] != None]

#now a function that will return both values with one call
def max_and_min(values):
    print('The lowest opening price was: ' + str(min(values)))
    print('The highest opening price was: ' + str(max(values)))

#find max and min for the list of opening prices 
max_and_min(opens)

The lowest opening price was: 34.0
The highest opening price was: 53.11


##### Task \#4: largest change in any one day based on high and low prices

In [9]:
#like before, find index of High and Low
def high_low_index():
    print('Index for High: ' + str(data_2017['dataset']['column_names'].index('High')))
    print('Index for Low: ' + str(data_2017['dataset']['column_names'].index('Low')))
    
high_low_index()

Index for High: 2
Index for Low: 3


In [10]:
#get a list of all 'High' prices
highs = [day[2] for day in daily] 

#similarly, get a list of 'Low' prices
lows = [day[3] for day in daily]

#now zip the hihgs and lows into a list of tuples
high_low_pairs = list(zip(highs, lows))

#create a list of differences between each high and low price
differences = [tup[0] - tup[1] for tup in high_low_pairs]

#calculate the largest change
print('The largest change in any one day was: ' + str(max(differences)))

The largest change in any one day was: 2.8100000000000023


##### Task 5: Largest change between any two days based on Closing Price

In [11]:
#index for 'Close' price
data_2017['dataset']['column_names'].index('Close')

4

In [12]:
#extract list of all closing prices
closes = [day[4] for day in daily]

#calculate amount of change from day to day and store in a new list
day_to_day = [closes[i] - closes[i-1] for i in range(1, len(closes))]
    
#calculate the absolute value of each change
day_to_day_abs = [abs(i) for i in day_to_day]

print('Largest change between any two days was: ' + str(max(day_to_day_abs)))

Largest change between any two days was: 2.559999999999995


##### Task 6: Average daily trading volume

In [13]:
#find index for traded volume
data_2017['dataset']['column_names'].index('Traded Volume')

6

In [14]:
#list of volumes
volume = [day[6] for day in daily]

#calculate the average
avg_volume = sum(volume) / len(volume)

print('The average volume traded was: ' + str(avg_volume))

The average volume traded was: 89124.33725490196


##### Task 7: median trading volume

In [15]:
import math

#sort the list of volumes
sort_vol = sorted(volume)

#basic function to calculate median, to be called on sorted list
def median(x):
    if len(x) %2 == 0:
        lower = int((len(x) / 2) - 1)
        upper = int(len(x) / 2)
        return (x[lower] + x[upper]) / 2
    elif len(x) %2 != 0:
        idx = math.ceil(len(x) / 2 - 1)
        return x[idx]
    
print('The median trading volume was: ' + str(median(sort_vol)))

The median trading volume was: 76286.0
